In [20]:
import numpy as np
import pandas as pd

np.random.seed(2019)

z1 = np.random.rand(100) * 10
z2 = np.random.rand(100) * 10

x1 = z1 + np.random.normal(0, 1, 100)
x2 = z1 + np.random.normal(0, 1, 100)
x3 = 2 * z1 + np.random.normal(0, 1, 100)
x4 = z2 + np.random.normal(0, 1, 100)
x5 = -3 * z2 + np.random.normal(0, 1, 100)


X = pd.DataFrame([x1, x2, x3, x4, x5]).T
Y = 2 * z1 - z2 + np.random.normal(0, 1, 100)



LASSO 回帰
scikit-learn パッケージの linear_model クラスで LASSO 回帰を行う Lasso メソッドが用意されている。このメソッドを使用して LASSO 回帰を行ってみる。LASSO 回帰を行うとき、正則化項の寄与の大きさを調整する正則化パラメーターをあらかじめ与える必要がある。このパラメーターを alpha 引数で指定する。まず、ここでは、仮に正則化パラメーターとして alpha = 0.1 と指定して回帰を行ってみる。

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

scaler = StandardScaler()
clf = Lasso(alpha=0.1)

scaler.fit(X)
clf.fit(scaler.transform(X), Y)
## Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
##    normalize=False, positive=False, precompute=False, random_state=None,
##    selection='cyclic', tol=0.0001, warm_start=False)

print(clf.coef_)
## [ 0.46288203  1.05025169  3.86729005 -0.84086274  2.24241141]

print(clf.intercept_)
## 5.666182032044889

[ 0.46288203  1.05025169  3.86729005 -0.84086274  2.24241141]
5.666182032044889


クロスバリデーション

最適な正則化パラメーターを決めるためにはクロスバリデーションを行う必要がある。scikit-learn パッケージの linear_model クラスにはクロスバリデーションと LASSO 回帰を同時に実行してくれる LassoCV メソッドが用意されている。ここで、最適な λ を 5-fold クロスバリデーションにより、10*-6 ≤ α ≤ 10*+2 の範囲内で 0.1 間隔のグリッドサーチを行なって決定する例を示す。

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

scaler = StandardScaler()
clf = LassoCV(alphas=10 ** np.arange(-6, 1, 0.1), cv=5)

scaler.fit(X)
clf.fit(scaler.transform(X), Y)
## LassoCV(alphas=array([1.00000e-06, 1.25893e-06, ..., 6.30957e+00, 7.94328e+00]),
##     copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
##     n_alphas=100, n_jobs=None, normalize=False, positive=False,
##     precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
##     verbose=False)

print(clf.alpha_)
## 0.005011872336272571

print(clf.coef_)
## [ 0.48686631  1.07953324  3.9316769  -0.92341824  2.27543548]

print(clf.intercept_)
## 5.666182032044889

0.005011872336272571
[ 0.48686631  1.07953324  3.9316769  -0.92341824  2.27543548]
5.666182032044889


モデル評価
モデルの作成からモデルの評価まで一通り実行する例を以下に示す。ここで、データセットの 8 割を教師データとして、LASSO 回帰を行う。このとき、最適な α を 5-fold クロスバリデーション決める。次に、教師データを除いた 2 割のデータを検証データとして　LASSO モデルを検証することにする。

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

scaler = StandardScaler()
clf = LassoCV(alphas=10 ** np.arange(-6, 1, 0.1), cv=5)

scaler.fit(x_train)
clf.fit(scaler.transform(x_train), y_train)

y_pred = clf.predict(scaler.transform(x_test))
mse = mean_squared_error(y_test, y_pred)
print(mse)
## 3.258166943313589

3.2581669433135865


In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()

ax = fig.add_subplot(1, 1, 1)
ax.scatter(y_pred, y_test)
ax.set_xlabel('predicted')
ax.set_ylabel('true')
ax.set_aspect('equal')

fig.show()

その他

In [17]:
import mglearn
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

X, y = mglearn.datasets.load_extended_boston()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

lasso = Lasso().fit(X_train, y_train)
print(f"training dataに対しての精度: {lasso.score(X_train, y_train):.2}")
print(f"test set scoreに対しての精度: {lasso.score(X_test, y_test):.2f}")
# => training dataに対しての精度: 0.29
# => test set scoreに対しての精度: 0.21

training dataに対しての精度: 0.29
test set scoreに対しての精度: 0.21
